<a href="https://colab.research.google.com/github/iwatake2222/tfjs_study/blob/master/mnist/CreateMnistModel_tfjs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KerasでMNIST数字識別モデルを作り、TensorFlow.jsモデルに変換する


## Kerasモデル作成、学習、h5モデル保存

In [0]:
%tensorflow_version 1.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np

print(tf.__version__)

# MNISTの学習用データ、テストデータをロードする
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# 学習データの前処理
# X: 6000x28x28x1のTensorに変換し、値を0～1.0に正規化
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_train = x_train / 255.
x_test = x_test / 255.
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

# Convolutionモデルの作成
input = tf.keras.layers.Input(shape=(28,28,1), name="input_image")
conv1 = tf.keras.layers.Conv2D(
  filters=8,
  kernel_size=(3,3),
  strides=(1,1),
  padding="same",
  activation="relu"
)(input)
pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(conv1)
conv2 = tf.keras.layers.Conv2D(
  filters=4,
  kernel_size=(3,3),
  strides=(1,1),
  padding="same",
  activation="relu"
)(pool1)
dropout1 = tf.keras.layers.Dropout(0.2)(conv2)
flatten1 = tf.keras.layers.Flatten()(dropout1)
output = tf.keras.layers.Dense(units=10, activation="softmax", name="output_scores")(flatten1)
model = tf.keras.models.Model(inputs=[input], outputs=[output], name="ConvMnist")
model.summary()

model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

# Convolutionモデルの学習
model.fit(
  x_train,
  y_train,
  batch_size=32,
  epochs=20,
  validation_split=0.2,
)

# 学習したモデルを使用して、テスト用データで評価する
model.evaluate(x_test,  y_test, verbose=2)

# 学習済みモデルをファイル(h5)に保存する
model.save("conv_mnist.h5")

1.15.0
11493376/11490434 [==============================] - 0s 0us/step
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "ConvMnist"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 8)         80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 4)         292       
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 4)         0         
_________________________________________________________________
flatten (Flatten)            (None, 

## Kerasモデル(h5)をTensorFlow.jsモデルに変換して、ローカルにダウンロード

In [0]:
!pip install tensorflowjs

In [0]:
!mkdir conv_mnist_tfjs
!tensorflowjs_converter --input_format keras \
                       conv_mnist.h5 \
                       ./conv_mnist_tfjs
!tar zcvf conv_mnist_tfjs.tar.gz conv_mnist_tfjs

# Download to local
from google.colab import files
files.download("./conv_mnist_tfjs.tar.gz")
files.download( "./conv_mnist.h5")

conv_mnist_tfjs/
conv_mnist_tfjs/group1-shard1of1.bin
conv_mnist_tfjs/model.json
